In [3]:
import tkinter as tk
from tkinter import simpledialog, messagebox
# import textdistance
import json
import csv

In [4]:
# Lectura de archivos y creación de diccionarios

with open("./Español.txt", "r", encoding="utf-8") as archivo1:
    contenido_archivo1 = archivo1.read().splitlines()
with open("./Ingles.txt", "r", encoding="utf-8") as archivo2:
    contenido_archivo2 = archivo2.read().splitlines()

diccionario_espanol_ingles = {palabra.lower(): traduccion.lower() for palabra, traduccion in zip(contenido_archivo1, contenido_archivo2)}
diccionario_ingles_espanol = {palabra.lower(): traduccion.lower() for palabra, traduccion in zip(contenido_archivo2, contenido_archivo1)}


In [5]:
root = tk.Tk()
root.title("Traductor")
root.geometry("350x200")

palabra_label = tk.Label(root, text="Palabra:")
palabra_entry = tk.Entry(root)
resultado_label = tk.Label(root, text="")

palabra_label.place(x=80, y=30)
palabra_entry.place(x=140, y=30)
resultado_label.place(x=40, y=100)



In [6]:
# Cargar o inicializar el archivo de frecuencias
def cargar_frecuencias():
    try:
        with open("frecuencias.json", "r", encoding="utf-8") as archivo:
            return json.load(archivo)
    except FileNotFoundError:
        return {}

def guardar_frecuencias(frecuencias):
    with open("frecuencias.json", "w", encoding="utf-8") as archivo:
        json.dump(frecuencias, archivo, ensure_ascii=False, indent=4)

In [7]:
def agregar_etiqueta(palabra, idioma_origen, etiquetas):


    if idioma_origen == 'español':
        palabra_traducida = simpledialog.askstring("Entrada", f"Por favor, introduce la traducción en inglés para '{palabra}':")
        gramatica = simpledialog.askstring("Entrada", f"Por favor, introduce la etiqueta gramatical para '{palabra}':")
        etiquetas[palabra] = (palabra_traducida, gramatica)
        etiquetas[palabra_traducida] = (palabra, gramatica)
    else:
        palabra_traducida = simpledialog.askstring("Entrada", f"Por favor, introduce la traducción en español para '{palabra}':")
        gramatica = simpledialog.askstring("Entrada", f"Por favor, introduce la etiqueta gramatical para '{palabra}':")
        etiquetas[palabra] = (palabra_traducida, gramatica)
        etiquetas[palabra_traducida] = (palabra, gramatica)

    with open('etiquetas.csv', 'a', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow([palabra, palabra_traducida, gramatica])

    return palabra_traducida, gramatica

In [8]:
def cargar_etiquetas_csv(archivo):
    etiquetas = {}
    with open(archivo, newline='', encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            if len(row) >= 3:  # Asegúrate de que la fila tenga al menos 3 elementos
                palabra_ingles = row[0]
                palabra_espanol = row[1]
                gramatica = row[2]
                etiquetas[palabra_ingles] = (palabra_espanol, gramatica)
                etiquetas[palabra_espanol] = (palabra_ingles, gramatica) 
    return etiquetas


In [9]:
def buscar_regla_gramatical(palabra, etiquetas,idioma_origen):

    if palabra in etiquetas:
        print(palabra)
        traduccion, gramatica = etiquetas[palabra]
        print(f'{traduccion} {gramatica}')
        return traduccion, gramatica
    else:
        messagebox.showinfo("Sin coincidencias", "Agregando la palabra manualmente.")
        traduccion, gramatica = agregar_etiqueta(palabra, idioma_origen, etiquetas)
        return traduccion, gramatica

        

In [10]:
def traducir_con_reglas(oracion, idioma_origen, etiquetas,archivo):

    palabras = oracion.split()

    traducion = []

    for palabra in palabras:
              
        palabra_traducida, gramatica = buscar_regla_gramatical(palabra, etiquetas,idioma_origen)
        etiquetas = cargar_etiquetas_csv(archivo)

        traducion.append(palabra_traducida)

    
    return traducion        

In [11]:

# Función para agregar una palabra manualmente al diccionario
def agregar_palabra_manualmente(palabra):
    traduccion_manual = simpledialog.askstring("Agregar palabra", f"Ingrese la traducción de '{palabra}':", parent=root).lower()
    if traduccion_manual:
        with open("./Español.txt", "a", encoding="utf-8") as archivo1:
            archivo1.write("\n" + palabra)
        with open("./Ingles.txt", "a", encoding="utf-8") as archivo2:
            archivo2.write("\n" + traduccion_manual)

        messagebox.showinfo("Palabra agregada", f"La traducción de '{palabra}' ha sido agregada correctamente.")

        diccionario_espanol_ingles[palabra] = traduccion_manual
        diccionario_ingles_espanol[traduccion_manual] = palabra

        if palabra_entry.get().lower() == palabra:
            resultado_label.config(text=f"La traducción de '{palabra}' es '{traduccion_manual}'.")


In [12]:
# Inicializar las etiquetas y las frecuencias
frecuencias = cargar_frecuencias()

# Función para sugerir una traducción basada en la entrada del usuario
def sugerir_traduccion(palabra, diccionario):

    sugerencias = []
    
    for palabra_dicc in diccionario:
        distancia = textdistance.levenshtein.normalized_distance(palabra, palabra_dicc)
        if distancia < 0.4:
            sugerencias.append((palabra_dicc, distancia))

    # Ordenar las sugerencias por frecuencia (descendente) y luego por distancia (ascendente)
    sugerencias.sort(key=lambda x: (-frecuencias.get(x[0], 0), x[1]))
    return sugerencias

In [13]:
# Función principal
def traducir_palabra():

    palabra = palabra_entry.get().lower()
    palabras = palabra.split()
    print(palabras)
    if len(palabras) < 2:

        if palabra in diccionario_espanol_ingles:
            traduccion = diccionario_espanol_ingles[palabra]
            resultado_label.config(text=f"La traducción de '{palabra}' es '{traduccion}'.")
        elif palabra in diccionario_ingles_espanol:

            traduccion = diccionario_ingles_espanol[palabra]
            resultado_label.config(text=f"La traducción de '{palabra}' es '{traduccion}'.")
        else:
            sugerencias_espanol = sugerir_traduccion(palabra, diccionario_espanol_ingles)
            sugerencias_ingles = sugerir_traduccion(palabra, diccionario_ingles_espanol)
        
            
            # Depuración: Mostrar sugerencias y frecuencias antes de preguntar al usuario
            print("Sugerencias ingles:", [(sug, frecuencias.get(sug, 0)) for sug, _ in sugerencias_ingles])
            print("Sugerencias español:", [(sug, frecuencias.get(sug, 0)) for sug, _ in sugerencias_espanol])


            if sugerencias_ingles or sugerencias_espanol:
                for sug, _ in sugerencias_ingles + sugerencias_espanol:

                    # Mostrar la frecuencia junto con la sugerencia
                    frecuencia = frecuencias.get(sug, 0)
                    respuesta = messagebox.askyesno("Sugerencia", f"¿Quisiste decir '{sug}'?")

                    if respuesta:
                        frecuencias[sug] = frecuencia + 1
                        guardar_frecuencias(frecuencias)
                        return
                messagebox.showinfo("Sin coincidencias", "Agregando la palabra manualmente.")
                agregar_palabra_manualmente(palabra)

            else:

                respuesta = messagebox.askyesno("No encontrada", f"La palabra '{palabra}' no se encontró en ninguno de los archivos.\n¿Desea agregarla manualmente?")
                
                if respuesta:
                    agregar_palabra_manualmente(palabra)

    elif len(palabras) >= 2: 
        
        primera_palabra = palabras[0]
       
        if primera_palabra in diccionario_espanol_ingles:
            traduccion = diccionario_espanol_ingles[primera_palabra]
            idioma_origen = 'español'
        elif primera_palabra in diccionario_ingles_espanol:
            traduccion = diccionario_ingles_espanol[primera_palabra]
            idioma_origen = 'ingles'
          
        print(f'Idioma Origen : {idioma_origen}')

        archivo_csv = 'etiquetas.csv'
        etiquetas = cargar_etiquetas_csv(archivo_csv)

        traduccions = traducir_con_reglas(palabra, idioma_origen, etiquetas,archivo_csv)
        oracion_traducida = " ".join(traduccions)
        resultado_label.config(text=f"La traducción de '{palabra}' es '{oracion_traducida}'.")
        
   
    

In [14]:

traducir_button = tk.Button(root, text="Traducir", command=traducir_palabra)
traducir_button.place(x=130, y=60)

root.mainloop()

['hola']
['salchicha']


Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\diego\AppData\Local\Programs\Python\Python311\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\diego\AppData\Local\Temp\ipykernel_32620\3787914139.py", line 17, in traducir_palabra
    sugerencias_espanol = sugerir_traduccion(palabra, diccionario_espanol_ingles)
                          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\diego\AppData\Local\Temp\ipykernel_32620\3672124269.py", line 10, in sugerir_traduccion
    distancia = textdistance.levenshtein.normalized_distance(palabra, palabra_dicc)
                ^^^^^^^^^^^^
NameError: name 'textdistance' is not defined


['salchicha']


Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\diego\AppData\Local\Programs\Python\Python311\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\diego\AppData\Local\Temp\ipykernel_32620\3787914139.py", line 17, in traducir_palabra
    sugerencias_espanol = sugerir_traduccion(palabra, diccionario_espanol_ingles)
                          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\diego\AppData\Local\Temp\ipykernel_32620\3672124269.py", line 10, in sugerir_traduccion
    distancia = textdistance.levenshtein.normalized_distance(palabra, palabra_dicc)
                ^^^^^^^^^^^^
NameError: name 'textdistance' is not defined


['salchicha']


Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\diego\AppData\Local\Programs\Python\Python311\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\diego\AppData\Local\Temp\ipykernel_32620\3787914139.py", line 17, in traducir_palabra
    sugerencias_espanol = sugerir_traduccion(palabra, diccionario_espanol_ingles)
                          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\diego\AppData\Local\Temp\ipykernel_32620\3672124269.py", line 10, in sugerir_traduccion
    distancia = textdistance.levenshtein.normalized_distance(palabra, palabra_dicc)
                ^^^^^^^^^^^^
NameError: name 'textdistance' is not defined


['salchicha']
['salchicha']


Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\diego\AppData\Local\Programs\Python\Python311\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\diego\AppData\Local\Temp\ipykernel_32620\3787914139.py", line 17, in traducir_palabra
    sugerencias_espanol = sugerir_traduccion(palabra, diccionario_espanol_ingles)
                          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\diego\AppData\Local\Temp\ipykernel_32620\3672124269.py", line 10, in sugerir_traduccion
    distancia = textdistance.levenshtein.normalized_distance(palabra, palabra_dicc)
                ^^^^^^^^^^^^
NameError: name 'textdistance' is not defined
Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\diego\AppData\Local\Programs\Python\Python311\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  F

['salchicha']
['salchicha']


Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\diego\AppData\Local\Programs\Python\Python311\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\diego\AppData\Local\Temp\ipykernel_32620\3787914139.py", line 17, in traducir_palabra
    sugerencias_espanol = sugerir_traduccion(palabra, diccionario_espanol_ingles)
                          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\diego\AppData\Local\Temp\ipykernel_32620\3672124269.py", line 10, in sugerir_traduccion
    distancia = textdistance.levenshtein.normalized_distance(palabra, palabra_dicc)
                ^^^^^^^^^^^^
NameError: name 'textdistance' is not defined
Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\diego\AppData\Local\Programs\Python\Python311\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  F